## GeoHash 튜토리얼
- [GeoHash 이론 참고자료](https://velog.io/@donghune/GeoHash-%EC%97%90-%EB%8C%80%ED%95%B4%EC%84%9C-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0)

In [1]:
import folium
import geohash2

# 중심 좌표 리스트
geographic_coordinates = [
    [37.5666805, 126.9784147], # latitude, longitude
    [37.565762249845776, 126.98707382055595],
    [37.570907047865816, 126.98814830055538],
    [37.5706978748617, 126.97551561217666]
]

cnt = 0
lat, lon = geographic_coordinates[cnt]

# 지도 크기
map_size = 15

# 지도 생성
m = folium.Map(location=[lat, lon], zoom_start=map_size)

### 지도 생성

In [2]:
# m

### GeoHash 및 위도 경도 범위 계산 후 지도에 반영
- 아래 코드를 반복 수행하여 geohash 영역이 지도에 순차적으로 반영되는 것을 확인
- 확인 결과 직방의 경우 기본 geohash 영역은 6, 최대 4까지 지원함을 확인
    - hash_value = geohash2.encode(lat, lon, precision=6) <- 4로 변경하면 최대 영역 확인 가능

In [16]:
def coord_to_geohash(lat, lon, precision=6):
    hash_value = geohash2.encode(lat, lon, precision)
    print(f"현재 입력된 위도: {lat}, 경도: {lon}")
    print(f"위도/경도 geohash 변환 결과: {hash_value}")
    print()

    return hash_value

def geohash_to_coord(hash_value):
    lat_center, lon_center, lat_err, lon_err = geohash2.decode_exactly(hash_value)
    lat_min = lat_center - lat_err
    lat_max = lat_center + lat_err
    lon_min = lon_center - lon_err
    lon_max = lon_center + lon_err
    coord = [lat_min, lat_max, lon_min, lon_max]

    print(f"{hash_value} 코드에 대한 위도 경도 범위")
    print(f"lat_min, lon_min: {lat_min}, {lon_min}")
    print(f"lat_max, lon_max: {lat_max}, {lon_max}")
    print()

    return coord

def draw_rect_to_map(lat, lon, precision):
    hash_value = coord_to_geohash(lat, lon, precision)
    coord = geohash_to_coord(hash_value)
    lat_min = coord[0]
    lat_max = coord[1]
    lon_min = coord[2]
    lon_max = coord[3]

    geohash_box = [(lat_min, lon_min), (lat_max, lon_max)]

    # Geohash 사각형 추가
    folium.Rectangle(
        bounds=geohash_box,
        color='blue',
        fill=True,
        fill_opacity=0.3
    ).add_to(m)

    # 중심점 마커
    folium.Marker([lat, lon], popup="Center").add_to(m)

In [10]:
lat, lon = geographic_coordinates[cnt]
draw_rect_to_map(lat, lon, precision=6)
cnt += 1

현재 입력된 위도: 37.5706978748617, 경도: 126.97551561217666
위도/경도 geohash 변환 결과: wydm9r

wydm9r 코드에 대한 위도 경도 범위
lat_min, lon_min: 37.5677490234375, 126.968994140625
lat_max, lon_max: 37.5732421875, 126.97998046875



In [11]:
m

## 크롤링 지역 범위 설정
- [GeoHash Explorer](https://geohash.softeng.co/wydj)
    - 해당 사이트를 통해 지도를 직접 보면서 geohash 영역 설정이 가능

## 직방 GeoHash 좌표 및 위도 경도 값 비교
- 원본 url : https://apis.zigbang.com/v2/items/villa?geohash=wydm3&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&salesTypes[2]=매매&salesPriceMin=0&lngEast=127.07335046775347&lngWest=126.99692402961125&latSouth=37.49835901597116&latNorth=37.523672523661446&domain=zigbang&checkAnyItemWithoutFilter=true
- url의 geohash 값과 함께 들어온 위도 경도 범위 값이 정확하게 일치하지 않으며 약간의 오차가 발생함

In [23]:
geohash_value = "wydm3"
coord = geohash_to_coord(geohash_value)
lat_min = coord[0]
lat_max = coord[1]
lon_min = coord[2]
lon_max = coord[3]

wydm3 코드에 대한 위도 경도 범위
lat_min, lon_min: 37.4853515625, 126.9580078125
lat_max, lon_max: 37.529296875, 127.001953125



In [24]:
lngEast=127.07335046775347
lngWest=126.99692402961125
latSouth=37.49835901597116
latNorth=37.523672523661446

print(lat_min - latSouth)
print(lat_max - latNorth)
print(lon_min - lngWest)
print(lon_max - lngEast)

-0.013007453471161057
0.005624351338553879
-0.03891621711125026
-0.0713973427534711


## 직방 빌라 매물 리스트 url 분석
- 원본 url : https://apis.zigbang.com/v2/items/villa?geohash=wydm3&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&salesTypes[2]=매매&salesPriceMin=0&lngEast=127.07335046775347&lngWest=126.99692402961125&latSouth=37.49835901597116&latNorth=37.523672523661446&domain=zigbang&checkAnyItemWithoutFilter=true
- 메인 주소 : https://apis.zigbang.com/v2/items/
    - v2는 매물이 표시되는 지도의 레벨을 의미
- 매물 조건 : villa?geohash=wydm3&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&salesTypes[2]=매매&salesPriceMin=0
    - vlila : 매물 종류를 입력, 아파트/오피스텔/공장 등 도 가능
    - geohash : 검색하려는 지역의 geohash 좌표를 입력
- 상세 좌표 : lngEast=127.07335046775347&lngWest=126.99692402961125&latSouth=37.49835901597116&latNorth=37.523672523661446
    - geohash의 좌표를 위도 경도 범위로 변환한 값
    - geohash 영역과 정확하게 일치하지 않음
    - 전체 url에서 이 부분의 코드는 삭제해도 동작됨을 확인함
        - 입력매물 조건의 geohash 입력만으로 해당 지역의 매물 리스트를 리스폰스 받을 수 있음
- 뭔지 모르는 정보 : domain=zigbang&checkAnyItemWithoutFilter=true